## import

In [2]:
import nibabel as nib
import numpy as np
import pandas as pd
import cortex
from statsmodels.stats.multitest import fdrcorrection
from nltools.stats import fdr, fisher_r_to_z, norm
from nilearn.glm import threshold_stats_img
from nilearn.image import smooth_img
from scipy.stats import zscore, pearsonr, ttest_1samp, ttest_rel, kendalltau, spearmanr
from scipy.ndimage import gaussian_filter

result_folder = '/home/dasom/SM_DATA/Distributed/files'

template = nib.load(f'{result_folder}/MNI_template_3mm.nii.gz')
roi_mask = nib.load(f'{result_folder}/BNA_3mm_atlas.nii.gz').get_fdata()
brain_mask = nib.load(f'{result_folder}/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()
gm_mask_path = f'{result_folder}/MNI152NLin2009cAsym_3mm_gray_mask.nii.gz'

gm_idx = np.where(roi_mask[brain_mask > 0] > 0)[0]
wm_idx = np.where(roi_mask[brain_mask > 0] == 0)[0]

def generating_plot(vol_data, path):
    cortex.export.save_3d_views(vol_data,
                        base_name = path,
                        list_angles = ['medial_pivot', 'lateral_pivot', 'bottom', 'back_pivot'],
                        list_surfaces = ['inflated', 'inflated', 'inflated', 'inflated'],
                        size=(1024 * 4, 768 * 2),
                        sleep=5)

In [4]:
template = nib.load(f'{result_folder}/MNI_template_3mm.nii.gz')
r = np.load(f'{result_folder}/VEM_f3_r_score0.npy')
p = np.load(f'{result_folder}/VEM_f3_p_value0.npy')
p_value_clip = np.clip(p, 0.0001, 0.9999)

z_map = norm.isf(p_value_clip)
z_map[z_map < 0] = 0
visualized_data = z_map
if len(visualized_data) == 69831:
    visualized_data[wm_idx] = 0

pycor_mask = np.full(brain_mask.shape, 0.0)
if len(visualized_data) == 69831:
    pycor_mask[brain_mask > 0] = visualized_data
else:
    pycor_mask[(brain_mask > 0) & (roi_mask > 0)] = visualized_data

new_img = nib.Nifti1Image(pycor_mask, affine=template.affine)

thresholded_img, voxel_thresh = threshold_stats_img(new_img, alpha = 0.05, height_control = 'fdr', two_sided = False, cluster_threshold = 10)

visualized_data = np.mean(r, axis = 0)
if len(visualized_data) == 69831:
    visualized_data[wm_idx] = np.nan 

pycor_mask = np.full(brain_mask.shape, np.nan)

if len(visualized_data) == 69831:
    pycor_mask[brain_mask > 0] = visualized_data
else:
    pycor_mask[(brain_mask > 0) & (roi_mask > 0)] = visualized_data

pycor_mask[thresholded_img.get_fdata() == 0] = np.nan

vol_data = cortex.Volume(pycor_mask.transpose(2,1,0), 'cvs_avg35_inMNI152', 'social_movie', cmap='plasma',
                        vmin = 0., vmax = 0.1
                        )
cortex.webshow(vol_data)

Started server on port 14775
Stopping server


<JS: window.viewer>

In [ ]:
generating_plot(vol_data, '/home/dasom/Dropbox/ubuntu-window/미팅자료/rating_recall/Results/Encoding_performance_full')

Started server on port 30151
{'camera.azimuth': 0, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
{'camera.azimuth': 180, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
{'camera.azimuth': 0, 'camera.altitude': 180, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 0, 'surface.{subject}.shift': 5, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
waiting for camera.altitude 179.9 -> 180
{'camera.azimuth': 180, 'camera.altitude': 60, 'camera.target': [0, -100, 0], 'surface.{subject}.unfol

: 

In [ ]:
a = np.load(f'{result_folder}/VEM_f3_r_score1.npy')
b = np.load(f'{result_folder}/VEM_f3_r_score2.npy')
p_a = np.load(f'{result_folder}/VEM_f3_p_value1.npy')
p_b = np.load(f'{result_folder}/VEM_f3_p_value2.npy')

template = nib.load('./mri_data/mask_files/MNI_template_3mm.nii.gz')
thresholded_imgs = []
for p in [p_a, p_b]:
    p_clip = np.clip(p, 0.0001, 0.9999)
    z_map = norm.isf(p_clip)
    visualized_data = z_map
    visualized_data[z_map < 0] = 0

    pycor_mask = np.full(brain_mask.shape, 0.0)
    pycor_mask[brain_mask > 0] = visualized_data

    new_img = nib.Nifti1Image(pycor_mask, affine=template.affine)

    thresholded_img, _ = threshold_stats_img(new_img, alpha = 0.05, height_control = 'fdr', 
                                                    two_sided = False, cluster_threshold = 10)
    thresholded_imgs.append(thresholded_img)

brain_mask = nib.load(f'{result_folder}/files/MNI152NLin2009cAsym_3mm_mask.nii.gz').get_fdata()

contrast = -0.01
visualized_data1 = np.mean(a, axis=0)
visualized_data2 = np.mean(b, axis=0)

score_brain1 = np.zeros(brain_mask.shape)
score_brain1[brain_mask > 0] = visualized_data2
score_brain1[thresholded_imgs[1].get_fdata() == 0] = contrast
# score_brain1[(brain_mask > 0) & (roi_mask > 0)] = visualized_data2

score_brain2 = np.zeros(brain_mask.shape)
score_brain2[brain_mask > 0] = visualized_data1
score_brain2[thresholded_imgs[0].get_fdata() == 0] = contrast
# score_brain2[(brain_mask > 0) & (roi_mask > 0)] = visualized_data1

vol_data = cortex.Volume2D(score_brain1.transpose(2,1,0), score_brain2.transpose(2,1,0), "cvs_avg35_inMNI152", "social_movie",
                           vmin = contrast, vmax = 0.08, vmin2 = contrast, vmax2 = 0.08,
                           cmap='MaCy_covar_alpha')
cortex.webshow(vol_data)

Started server on port 50422
Stopping server


<JS: window.viewer>

In [ ]:
generating_plot(vol_data, '/home/dasom/Dropbox/ubuntu-window/미팅자료/rating_recall/Results/Encoding_performance_cooccur_vs_valence')

Started server on port 55143
{'camera.azimuth': 0, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
{'camera.azimuth': 180, 'camera.altitude': 90, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 180, 'surface.{subject}.shift': 10, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
{'camera.azimuth': 0, 'camera.altitude': 180, 'camera.target': [0, 0, 0], 'surface.{subject}.unfold': 1.0, 'surface.{subject}.pivot': 0, 'surface.{subject}.shift': 5, 'surface.{subject}.specularity': 0, 'surface.{subject}.sampler': 'nearest', 'surface.{subject}.layers': 1}
waiting for camera.altitude 179.9 -> 180
{'camera.azimuth': 180, 'camera.altitude': 60, 'camera.target': [0, -100, 0], 'surface.{subject}.unfol

: 

## End